In [4]:
# Cell 1: Clone repo and install dependencies
import os

REPO_URL = "https://github.com/elonmj/Code-traffic-flow.git"
REPO_DIR = "/content/Code-traffic-flow"

if os.path.exists(REPO_DIR):
    print(f"Repository already exists at {REPO_DIR}")
    %cd {REPO_DIR}
    !git pull
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

!pip install stable-baselines3 gymnasium numba --quiet
print(f"✅ Setup complete | Working dir: {os.getcwd()}")

Cloning into '/content/Code-traffic-flow'...
remote: Enumerating objects: 8994, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 8994 (delta 114), reused 160 (delta 61), pack-reused 8772 (from 2)
Receiving objects: 100% (8994/8994), 314.68 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (4610/4610), done.
/content/Code-traffic-flow
✅ Setup complete | Working dir: /content/Code-traffic-flow


In [5]:
# Cell 2: Imports
import sys
sys.path.insert(0, REPO_DIR)

import numpy as np
import torch
import time
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback

from arz_model.config import create_victoria_island_config
from Code_RL.src.env.traffic_signal_env_direct_v3 import TrafficSignalEnvDirectV3

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch: 2.9.0+cu126
CUDA: True
GPU: Tesla T4


In [6]:
# Cell 3: Configuration
DEFAULT_DENSITY = 120.0  # Congested scenario
INFLOW_DENSITY = 180.0
REWARD_WEIGHTS = {'alpha': 5.0, 'kappa': 0.3, 'mu': 0.1}

def create_env(quiet=True):
    """Create traffic environment"""
    config = create_victoria_island_config(
        t_final=450.0, output_dt=15.0, cells_per_100m=4,
        default_density=DEFAULT_DENSITY, inflow_density=INFLOW_DENSITY, use_cache=False
    )
    config.rl_metadata = {'observation_segment_ids': [s.id for s in config.segments], 'decision_interval': 15.0}
    
    class SimpleConfig:
        def __init__(self, c):
            self.arz_simulation_config = c
            self.rl_env_params = {'dt_decision': 15.0, 'observation_segment_ids': None, 'reward_weights': REWARD_WEIGHTS}
    
    return TrafficSignalEnvDirectV3(
        simulation_config=SimpleConfig(config).arz_simulation_config,
        decision_interval=15.0, reward_weights=REWARD_WEIGHTS, quiet=quiet
    )

print(f"✅ Environment factory ready (density={DEFAULT_DENSITY}, inflow={INFLOW_DENSITY})")

✅ Environment factory ready (density=120.0, inflow=180.0)


In [7]:
# Cell 4: Evaluate Baselines (Random, FT-30s, FT-60s, FT-90s)
env = create_env()
baseline_results = {}

def eval_fixed_time(env, interval, n_ep=3):
    rewards = []
    for _ in range(n_ep):
        obs, _ = env.reset()
        done, ep_r, t = False, 0.0, 0.0
        while not done:
            t += env.decision_interval
            action = 1 if t >= interval else 0
            if action == 1: t = 0.0
            obs, r, done, _, _ = env.step(action)
            ep_r += r
        rewards.append(ep_r)
    return {'mean_reward': np.mean(rewards), 'std_reward': np.std(rewards)}

# Random
print("🎲 Random...")
rnd = [sum([env.step(env.action_space.sample())[1] for _ in range(30)]) for _ in [env.reset() for _ in range(3)]]
baseline_results['Random'] = {'mean_reward': np.mean(rnd), 'std_reward': np.std(rnd)}

# Fixed-time baselines
for name, interval in [('FT-30s', 30), ('FT-60s', 60), ('FT-90s', 90)]:
    print(f"⏱️ {name}...")
    baseline_results[name] = eval_fixed_time(env, interval)

print("\n📊 BASELINES:")
for n, d in sorted(baseline_results.items(), key=lambda x: x[1]['mean_reward'], reverse=True):
    print(f"  {n:10s}: {d['mean_reward']:>8.1f} ± {d['std_reward']:.1f}")


🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   🚦 Detected 8 signalized nodes from OSM data

   🔧 Generating segment configurations...
   ✅ Created 70 segment configurations

   🔧 Generating node configurations...
   ✅ Created 60 node configurations

   ⚙️  Setting up time and physics parameters...

   🔨 Assembling complete network configuration...


   🔧 Generating segment configurations...
   ✅ Created 70 segment configurations

   🔧 Generating node configurations...
   ✅ Created 60 node configurations

   ⚙️  Setting up time and physics parameters...

   🔨 Assembling complete network configuration...

✅ CONFIGURATION GENERATION COMPLETE
   Total Segments: 70
   Total Nodes: 60
   Entry Points: 4
   Exit Points: 4
   Junctions: 15
   Signalized Nodes: 8
✅ CONFIGURATION GENERATION COMPLETE
   Total Segments: 70
   Total Nodes: 60
   Entry Points: 4
   Exit Points: 4
   Junctions: 15
   Signalized Nodes: 8
   Simulation Duration: 450.0s (7.5 min)
   Grid Resolution: 4 cells/100m

[NETWORK BUILD] Creating 70 segments and 60 nodes.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   [10/70] segments created
   [20/70] segments created
   [30/70] segments created
   [40/70] segments created
   [50/70] segments created
   [60/70] segments created
   [70/70] segments created
[NETWORK BUILD] ✅ Network construction complete!
   Total segments: 70
   Total nodes: 60
Finalizing network structure and validating topology...
✅ Network topology is valid.


...

  - Preparing GPU topology for network coupling...
    - GPU topology prepared and transferred.
🎲 Random...


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size

[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_bound

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid siz

[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_boundary_phases_bulk] Updating 16 segments on GPU, 0 not found in pool
[DEBUG set_bound

In [ ]:
# Cell 5: Progressive Training Callback
class ProgressCallback(BaseCallback):
    def __init__(self, eval_env, ref_reward, target_pct=10.0, eval_freq=1000, n_eval=3):
        super().__init__()
        self.eval_env, self.ref = eval_env, ref_reward
        self.target = ref_reward * (1 + target_pct/100)
        self.eval_freq, self.n_eval = eval_freq, n_eval
        self.history, self.best, self.reached = [], -np.inf, False
        
    def _on_step(self):
        if self.n_calls % self.eval_freq == 0:
            rewards = []
            for _ in range(self.n_eval):
                obs, done, r = self.eval_env.reset()[0], False, 0
                while not done:
                    a, _ = self.model.predict(obs, deterministic=True)
                    obs, rew, done, _, _ = self.eval_env.step(a)
                    r += rew
                rewards.append(r)
            mean_r = np.mean(rewards)
            imp = ((mean_r - self.ref) / abs(self.ref)) * 100
            self.history.append({'step': self.num_timesteps, 'reward': mean_r, 'improvement': imp})
            if mean_r > self.best: self.best = mean_r
            status = "🎯" if imp >= 10 else ""
            print(f"  [{self.num_timesteps:>6}] R={mean_r:>7.1f} | vs FT-90s: {imp:>+5.1f}% {status}")
            if imp >= 10 and not self.reached:
                self.reached = True
                print(f"\n🏆 TARGET REACHED!")
        return True

FT90_REF = baseline_results['FT-90s']['mean_reward']
print(f"✅ Callback ready | Reference: FT-90s = {FT90_REF:.1f} | Target: {FT90_REF*1.1:.1f}")

In [ ]:
# Cell 6: Initialize DQN Model
train_env = create_env()
eval_env = create_env()

model = DQN("MlpPolicy", train_env, learning_rate=1e-4, buffer_size=50000,
            learning_starts=1000, batch_size=64, tau=0.005, gamma=0.99,
            exploration_fraction=0.3, exploration_final_eps=0.05,
            verbose=0, device='cuda' if torch.cuda.is_available() else 'cpu')

callback = ProgressCallback(eval_env, FT90_REF, target_pct=10.0, eval_freq=1000)
STATE = {'model': model, 'callback': callback, 'steps': 0, 'block': 0, 'done': False}

print(f"🚀 DQN ready on {model.device}")

In [ ]:
# Block 1: Steps 0 → 10,000
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 1: 0 → 10k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 1, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_10k")
    print(f"✅ Block 1 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 2: Steps 10k → 20k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 2: 10k → 20k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 2, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_20k")
    print(f"✅ Block 2 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 3: Steps 20k → 30k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 3: 20k → 30k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 3, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_30k")
    print(f"✅ Block 3 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 4: Steps 30k → 40k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 4: 30k → 40k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 4, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_40k")
    print(f"✅ Block 4 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 5: Steps 40k → 50k (HALFWAY)
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 5: 40k → 50k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 5, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_50k")
    imp = ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100
    print(f"📈 HALFWAY: {STATE['steps']} steps | Best improvement: {imp:+.1f}%")

In [ ]:
# Block 6: Steps 50k → 60k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 6: 50k → 60k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 6, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_60k")
    print(f"✅ Block 6 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 7: Steps 60k → 70k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 7: 60k → 70k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 7, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_70k")
    print(f"✅ Block 7 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 8: Steps 70k → 80k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 8: 70k → 80k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 8, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_80k")
    print(f"✅ Block 8 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 9: Steps 80k → 90k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 9: 80k → 90k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 9, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_90k")
    print(f"✅ Block 9 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 10: Steps 90k → 100k (FINAL)
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 10 (FINAL): 90k → 100k")
    STATE['model'].learn(10000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 10, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save("/content/model_FINAL")

imp = ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100
print(f"\n{'='*60}")
print(f"🏁 TRAINING COMPLETE")
print(f"   Total steps: {STATE['steps']}")
print(f"   Best reward: {STATE['callback'].best:.1f}")
print(f"   Improvement vs FT-90s: {imp:+.1f}%")
print(f"   Target (+10%): {'✅ ACHIEVED' if STATE['done'] else '❌ NOT ACHIEVED'}")
print(f"{'='*60}")

In [ ]:
# Final: Plot & Save Results
import matplotlib.pyplot as plt
import json

h = STATE['callback'].history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot([x['step'] for x in h], [x['reward'] for x in h], 'b-o', markersize=3)
ax1.axhline(y=FT90_REF, color='r', linestyle='--', label='FT-90s')
ax1.axhline(y=baseline_results['FT-30s']['mean_reward'], color='g', linestyle=':', label='FT-30s')
ax1.set_xlabel('Steps'); ax1.set_ylabel('Reward'); ax1.legend(); ax1.set_title('Learning Curve')

ax2.plot([x['step'] for x in h], [x['improvement'] for x in h], 'b-o', markersize=3)
ax2.axhline(y=10, color='g', linestyle='--', label='Target +10%')
ax2.axhline(y=0, color='r', linestyle='-', alpha=0.5)
ax2.set_xlabel('Steps'); ax2.set_ylabel('Improvement (%)'); ax2.legend(); ax2.set_title('Improvement vs FT-90s')

plt.tight_layout()
plt.savefig('/content/stage3_results.png', dpi=150)
plt.show()

# Save JSON
results = {'baselines': baseline_results, 'history': h, 'best': STATE['callback'].best, 
           'improvement': ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100}
with open('/content/stage3_results.json', 'w') as f: json.dump(results, f, indent=2)
print("📁 Saved: stage3_results.png, stage3_results.json")